In [2]:
import pandas as pd
import numpy as np
import os
import glob

def load_and_merge_data(folder_path):
    all_files = glob.glob(os.path.join(folder_path, "*.csv"))
    df_list = []
    sector_map = {} # Pour garder en mémoire quel actif appartient à quel secteur (utile pour Streamlit plus tard)
    print(f"Trouvé {len(all_files)} fichiers. Fusion en cours...")

    for filename in all_files:
        # Récupérer le nom du secteur depuis le nom du fichier (ex: 'Energy.csv' -> 'Energy')
        sector_name = os.path.basename(filename).replace('.csv', '')
        
        # 2. Lire le CSV en mettant la Date en index directement
        # parse_dates=True permet de comprendre que c'est une date temporelle
        df_temp = pd.read_csv(filename, parse_dates=['Date'], index_col='Date')
        
        # Petit nettoyage : on s'assure que ce sont des nombres
        df_temp = df_temp.apply(pd.to_numeric, errors='coerce')
        
        # Sauvegarder le secteur de chaque actif (Bonus pour la suite du projet)
        for asset in df_temp.columns:
            sector_map[asset] = sector_name
            
        df_list.append(df_temp)

    # 3. Concaténation horizontale (axis=1)
    # Pandas aligne tout seul sur l'index 'Date'. Pas de doublons de colonne Date.
    full_price_data = pd.concat(df_list, axis=1)
    
    # 4. Nettoyage des données manquantes (Indispensable pour l'optimisation)
    # Si un actif a des trous, on remplit avec la valeur précédente (ffill) puis on supprime les lignes vides restantes
    full_price_data = full_price_data.ffill().dropna()

    return full_price_data, sector_map

# Chemin vers ton dossier 'data' (adapte le chemin si besoin)
folder_path = '../data' 

# 1. Récupération des prix fusionnés
prices_df, sector_info = load_and_merge_data(folder_path)

print("\n--- Résumé des données chargées ---")
print(f"Nombre d'actifs (N) : {prices_df.shape[1]}")
print(f"Nombre de jours (T) : {prices_df.shape[0]}")
print("Aperçu des 3 premières lignes :")
print(prices_df.head(3))

# --- PRÉPARATION POUR L'ALGO (Partie 1) ---

# 2. Calcul des Rendements (Returns)
# L'optimisation de Markowitz se fait sur les rendements, pas sur les prix bruts.
# pct_change() calcule (Prix_t - Prix_t-1) / Prix_t-1
returns_df = prices_df.pct_change().dropna()

# 3. Calcul des inputs mathématiques pour ton algo convexe
mu = returns_df.mean().values  # Vecteur des rendements moyens espérés
Sigma = returns_df.cov().values # Matrice de covariance

print("\n--- Prêt pour l'optimisation ---")
print(f"Forme de mu : {mu.shape}")
print(f"Forme de Sigma : {Sigma.shape}")

Trouvé 11 fichiers. Fusion en cours...

--- Résumé des données chargées ---
Nombre d'actifs (N) : 197
Nombre de jours (T) : 1120
Aperçu des 3 premières lignes :
                GOOGL        META         DIS       NFLX         VZ  \
Date                                                                  
2020-09-16  75.086624  261.905426  129.924072  48.386002  44.270195   
2020-09-17  73.842697  253.258759  128.084747  47.020000  44.431526   
2020-09-18  72.057510  250.982758  126.520805  46.995998  44.255527   

                    T        TMUS      CMCSA        CHTR          EA  ...  \
Date                                                                  ...   
2020-09-16  15.401609  109.904205  40.505207  630.070007  123.228508  ...   
2020-09-17  15.312070  108.116905  39.440876  620.309998  121.789543  ...   
2020-09-18  15.238322  106.967239  39.163971  625.250000  122.411789  ...   

                 WELL        EQIX        NEE        DUK         SO          D  \
Date            

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sco
import glob
import os

# ==========================================
# 1. CHARGEMENT ET FUSION DES DONNÉES
# ==========================================
def load_data(folder_path='data'):
    print("--- Chargement des fichiers ---")
    all_files = glob.glob(os.path.join(folder_path, "*.csv"))
    
    if not all_files:
        raise FileNotFoundError(f"Aucun fichier CSV trouvé dans '{folder_path}'")
        
    df_list = []
    
    for filename in all_files:
        # Lire le fichier, mettre la Date en index
        df = pd.read_csv(filename, parse_dates=['Date'], index_col='Date')
        
        # Nettoyage : conversion en numérique au cas où
        df = df.apply(pd.to_numeric, errors='coerce')
        
        # On ajoute au tableau
        df_list.append(df)
        print(f"Chargé : {os.path.basename(filename)} ({df.shape[1]} actifs)")

    # Fusion sur l'index (Date)
    full_df = pd.concat(df_list, axis=1)
    
    # Nettoyage des données manquantes (Forward Fill puis Drop)
    full_df = full_df.ffill().dropna()
    
    print(f"\nDonnées fusionnées : {full_df.shape[1]} actifs sur {full_df.shape[0]} jours.")
    return full_df

# ==========================================
# 2. PRÉPARATION MATHEMATIQUE (Mu & Sigma)
# ==========================================

# Calcul des Rendements (Returns) : (Prix_t - Prix_t-1) / Prix_t-1
returns_df = prices_df.pct_change().dropna()

# Vectorisation pour l'optimiseur
mu = returns_df.mean().values * 252  # Annualisation (facultatif mais standard)
Sigma = returns_df.cov().values * 252 # Annualisation

n_assets = len(mu)
asset_names = returns_df.columns.tolist()

# ==========================================
# 3. MOTEUR D'OPTIMISATION CONVEXE (SCIPY)
# ==========================================

def portfolio_stats(weights, mu, Sigma):
    """Retourne (Rendement, Risque/Variance)"""
    p_ret = np.dot(weights, mu)
    # Formule du risque quadratique w^T * Sigma * w [cite: 84]
    p_var = np.dot(weights.T, np.dot(Sigma, weights)) 
    return p_ret, p_var

def scalarized_objective(weights, mu, Sigma, lambd):
    """
    Fonction objectif scalaire 
    Minimiser: lambda * Risque - (1 - lambda) * Rendement
    """
    ret, var = portfolio_stats(weights, mu, Sigma)
    # On minimise, donc on inverse le signe du rendement
    return lambd * var - (1 - lambd) * ret

# Configuration du solveur
bounds = tuple((0, 1) for _ in range(n_assets)) # Pas de vente à découvert (Poids >= 0) [cite: 107]
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) # Somme des poids = 1 [cite: 104]
init_guess = np.array([1/n_assets] * n_assets) # Départ équiréparti

# ==========================================
# 4. CALCUL DE LA FRONTIÈRE EFFICIENTE
# ==========================================
print("\n--- Optimisation en cours (Scalarisation) ---")

pareto_risks = []
pareto_returns = []
pareto_weights = []

# On balaie lambda de 0 (Max Rendement) à 1 (Min Risque)
lambdas = np.linspace(0, 1, 50) 

for l in lambdas:
    res = sco.minimize(
        scalarized_objective,
        init_guess,
        args=(mu, Sigma, l),
        method='SLSQP', # Solveur pour problèmes convexes contraints [cite: 1018]
        bounds=bounds,
        constraints=constraints
    )
    
    if res.success:
        r, v = portfolio_stats(res.x, mu, Sigma)
        pareto_returns.append(r)
        pareto_risks.append(v)
        pareto_weights.append(res.x)

print("Optimisation terminée.")

# ==========================================
# 5. VISUALISATION (LIVRABLES)
# ==========================================

# Graphique 1 : Frontière Efficiente
plt.figure(figsize=(12, 6))
plt.scatter(pareto_risks, pareto_returns, c=lambdas, cmap='viridis', marker='o')
plt.title('Frontière Efficiente (Modèle de Markowitz Convexe)')
plt.xlabel('Risque (Variance Portefeuille) $f_2$')
plt.ylabel('Rendement Espéré (Annualisé) $f_1$')
plt.colorbar(label='Paramètre de préférence $\lambda$ (0=Audacieux, 1=Prudent)')
plt.grid(True, linestyle='--', alpha=0.5)

# Afficher les points extrêmes
min_risk_idx = np.argmin(pareto_risks)
max_ret_idx = np.argmax(pareto_returns)
plt.scatter(pareto_risks[min_risk_idx], pareto_returns[min_risk_idx], c='red', s=200, marker='*', label='Min Variance')
plt.scatter(pareto_risks[max_ret_idx], pareto_returns[max_ret_idx], c='green', s=200, marker='*', label='Max Rendement')
plt.legend()
plt.show()

# Graphique 2 : Composition (Stacked Plot)
# Attention: Avec beaucoup d'actifs, ce graphe est dense. On affiche tout de même.
weights_df = pd.DataFrame(pareto_weights, columns=asset_names)
weights_df['Risque'] = pareto_risks
weights_df = weights_df.sort_values('Risque')

# Pour la lisibilité, on ne garde que les actifs qui dépassent 1% du portefeuille à un moment donné
active_assets = weights_df.drop('Risque', axis=1).max() > 0.01
filtered_weights = weights_df.loc[:, active_assets]
remaining_weights = 1 - filtered_weights.sum(axis=1)

plt.figure(figsize=(12, 6))
plt.stackplot(weights_df['Risque'], filtered_weights.T, labels=filtered_weights.columns, alpha=0.8)
plt.title('Allocation d\'actifs le long de la frontière (Actifs principaux > 1%)')
plt.xlabel('Niveau de Risque')
plt.ylabel('Proportion dans le portefeuille')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1)
plt.tight_layout()
plt.show()


--- Optimisation en cours (Scalarisation) ---
